<a href="https://colab.research.google.com/github/shuchimishra/Model_deployment_using_Kubernetes_and_Docker/blob/main/Running_Flask_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp -r "/content/drive/MyDrive/Data Science & Machine Learning/MLOps/Repository/model_output" "/content"

In [3]:
!ls "/content/model_output"

sa_encoder.vocab.tokens  sentiment_analysis.hdf5


In [4]:
import socket
print(socket.gethostbyname(socket.gethostname()))

172.28.0.12


In [5]:
import tensorflow_datasets as tfds
import tensorflow as tf
from flask import Flask, jsonify, make_response, request
import threading

app = Flask(__name__)
padding_size = 1000
model = tf.keras.models.load_model('/content/model_output/sentiment_analysis.hdf5')
text_encoder = tfds.deprecated.text.TokenTextEncoder.load_from_file("/content/model_output/sa_encoder.vocab")

print('Model and Vocabalory loaded.......')

@app.route("/")
def hello():
    return "I am alive!"

def pad_to_size(vec, size):
    zeros = [0] * (size - len(vec))
    vec.extend(zeros)
    return vec


def predict_fn(predict_text, pad_size):
    encoded_text = text_encoder.encode(predict_text)
    encoded_text = pad_to_size(encoded_text, pad_size)
    encoded_text = tf.cast(encoded_text, tf.int64)
    predictions = model.predict(tf.expand_dims(encoded_text, 0))

    return (predictions.tolist())


@app.route('/seclassifier', methods=['POST'])
def predict_sentiment():
    text = request.get_json()['text']
    print(text)
    predictions = predict_fn(text, padding_size)
    sentiment = 'positive' if float(''.join(map(str,predictions[0]))) > 0 else 'Negative'
    return jsonify({'predictions ':predictions, 'sentiment ': sentiment})


threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':8000}).start()

Model and Vocabalory loaded.......
 * Serving Flask app '__main__'


In [6]:
import requests
req = requests.get("http://172.28.0.12:8000/")
print(req.status_code)
print(req.text)

 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.28.0.12:8000
INFO:werkzeug:Press CTRL+C to quit


ConnectionError: HTTPConnectionPool(host='172.28.0.12', port=8000): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c161c30e050>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
seclassifier = "http://172.28.0.12:8000/seclassifier"

In [ ]:
import requests, json
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
data = {"text":"Still working my way through it but definitely changes your view on investment. Wish it was available on Audible"}
req = requests.post(seclassifier,  data=json.dumps(data), headers=headers)

print(req.status_code)
print(req.text)

In [ ]:
import requests, json
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
data = {"text":"I thought this will be good one. But I was wrong"}
req = requests.post(seclassifier,  data=json.dumps(data), headers=headers)

print(req.status_code)
print(req.text)

In [ ]:
!ps -eaf | grep python

In [ ]:
import flask
flask.__version__

In [ ]:
!pip install flask-ngrok

In [ ]:
!pip install pyngrok==4.1.1

In [ ]:
ngrok_token = "2i1tR1r0dWSLSjuyO8EUwxjiSN6_64ZXS9881Ldxy646BpSb4" # @param {type:"string"}


In [ ]:
!cat /root/.ngrok2/ngrok.yml

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
from flask import Flask, jsonify, make_response, request
from flask_ngrok import run_with_ngrok
!ngrok authtoken `echo $ngrok_token`


app = Flask(__name__)
run_with_ngrok(app)

padding_size = 1000
model = tf.keras.models.load_model('/content/model_output/sentiment_analysis.hdf5')
text_encoder = tfds.deprecated.text.TokenTextEncoder.load_from_file("/content/model_output/sa_encoder.vocab")

print('Model and Vocabalory loaded.......')

@app.route("/")
def hello():
    return "I am alive!"

def pad_to_size(vec, size):
    zeros = [0] * (size - len(vec))
    vec.extend(zeros)
    return vec


def predict_fn(predict_text, pad_size):
    encoded_text = text_encoder.encode(predict_text)
    encoded_text = pad_to_size(encoded_text, pad_size)
    encoded_text = tf.cast(encoded_text, tf.int64)
    predictions = model.predict(tf.expand_dims(encoded_text, 0))

    return (predictions.tolist())


@app.route('/seclassifier', methods=['POST'])
def predict_sentiment():
    text = request.get_json()['text']
    print(text)
    predictions = predict_fn(text, padding_size)
    sentiment = 'positive' if float(''.join(map(str,predictions[0]))) > 0 else 'Negative'
    return jsonify({'predictions ':predictions, 'sentiment ': sentiment})

if __name__ == '__main__':
    app.run()